# Taller de modelación de tsunamis con GeoCLAW

## Introducción


El objetivo de este taller es desarrollar un ejemplo de aplicación de modelación de tsunamis con *GeoCLAW*, autocontenido en un cuaderno de *Jupyter*  en cuanto al procesamiento de datos, simulación computacional, procesamiento de resultados y generación de gráficos, que le sirva al alumno como punto de partida al realizar simulaciones más complejas. Un objetivo secundario, es promover el desarrollo de resultados fáciles de compartir, bien documentados y reproducibles, mediante herramientas gratuitas y de código abierto como *IPython* y los cuadernos de *Jupyter* :).

GeoClaw es parte de [ClawPack](https://www.clawpack.org), un software gratuito de código abierto desarrollado en la Universidad de Washington que consta de varios paquetes computacionales que mediante métodos numéricos integran **leyes de conservación**. Geoclaw en particular está destinado a resolver problemas en Geofísica, como la simulación de tsunamis, *storm-surges*, y otros.

En nuestro caso, permite resolver las **ecuaciones de aguas someras** (St. Venant 2D) considerando **variaciones fuertes en la topografía**, **fricción con el fondo**, y manejando naturalmente **ondas de choque** (discontinuidades) mediante la solución al **problema de Riemann** entre cada celda. Además, para mejorar el rendimiento de la simulación, dispone de **mallas adaptativas**, que permiten variar la discretización espacial a lo largo del tiempo de la simulación. Si no entiendes algunos de estos conceptos que he puesto en **negrita**, te recomiendo que los investigues. Existe excelente material gratuito para estudiar en internet, como por ejemplo los del [curso PASI-TSUNAMI realizado el año 2013 en la UTFSM](http://www.bu.edu/pasi-tsunami/materials/).

### Cómo trabajar con los cuadernos de Jupyter

Los cuadernos de Jupyter (como este) son documentos que se editan en el explorador de internet y dividen su contenido en celdas, las que pueden contener código de algún lenguaje de programación o texto formateado. En nuestro caso programaremos siempre en python 2.7. Además, posee una interfaz gráfica (barra superior) que permite manejar las celdas pero lo más recomendado es utilizar atajos de teclado para trabajar con más fluidez. Estos los pueden ver [en la documentación](https://ipython.org/ipython-doc/3/notebook/notebook.html#basic-workflow). Los que más utilizo son:

* ctrl+enter = ejecutar celda actual
* esc y luego a = agregar celda arriba
* esc y luego b = agregar celda abajo
* esc y luego d = eliminar celda
* esc y luego m = cambiar celda a modo texto con formato (markdown)

### Estructura del cuaderno

En primer lugar discutiremos sobre cómo GeoClaw lee la **topografía**. Luego veremos cómo calcular la deformación de la superficie libre del agua que se genera por la **falla tectónica** asociada a un terremoto en el océano. Antes de hacer los cálculos, configuraremos GeoClaw para que guarde las **alturas máximas y tiempos de arribo** durante la simulación, con lo que ya estaremos listos para **configurar** y  **correr la simulación**. Finalmente, veremos cómo obtener **mapas de propagación y resultados puntuales** como la evolucion de la altura de la superficie libre en una boya virtual o real.

### Aclaración

El contenido de este cuaderno es una versión modificada de la demostración disponible en el [sitio web de ClawPack](http://www.clawpack.org/gallery/gallery_geoclaw.html).

## La topografía

Download data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
from clawpack.geoclaw import topotools
import clawpack.clawutil.data as clawutildata
topo_fname = 'etopo10min120W60W60S0S.asc'
url = 'http://www.geoclaw.org/topo/etopo/' + topo_fname
clawutildata.get_remote_file(url, output_dir='.', file_name=topo_fname,
        verbose=True)

Draw it

In [ ]:
topo = topotools.Topography(topo_fname, topo_type=2)
topo.plot()

## Traspasando la falla a la superficie libre

Set fault parameters

In [ ]:
from clawpack.geoclaw import dtopotools

# Specify subfault parameters for this simple fault model consisting
# of a single subfault:
usgs_subfault = dtopotools.SubFault()
usgs_subfault.strike = 16.
usgs_subfault.length = 450.e3
usgs_subfault.width = 100.e3
usgs_subfault.depth = 35.e3
usgs_subfault.slip = 15.
usgs_subfault.rake = 104.
usgs_subfault.dip = 14.
usgs_subfault.longitude = -72.668
usgs_subfault.latitude = -35.826
usgs_subfault.coordinate_specification = "top center"

fault = dtopotools.Fault()
fault.subfaults = [usgs_subfault]

print "Mw = ",fault.Mw()

Descargar

In [ ]:
import os
dtopo_fname = os.path.join('.', "dtopo_usgs100227.tt3")

In [ ]:
if os.path.exists(dtopo_fname):
    print "*** Not regenerating dtopo file (already exists): %s" \
                % dtopo_fname
else:
    print "Using Okada model to create dtopo file"

    x = np.linspace(-77, -67, 100)
    y = np.linspace(-40, -30, 100)
    times = [1.]

    fault.create_dtopography(x,y,times)
    dtopo = fault.dtopo
    dtopo.write(dtopo_fname, dtopo_type=3)

Now plot.

In [ ]:
if fault.dtopo is None:
    # read in the pre-existing file:
    print "Reading in dtopo file..."
    dtopo = dtopotools.DTopography()
    dtopo.read(dtopo_fname, dtopo_type=3)
    x = dtopo.x
    y = dtopo.y
plt.figure(figsize=(10,5))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
fault.plot_subfaults(axes=ax1,slip_color=True)
ax1.set_xlim(x.min(),x.max())
ax1.set_ylim(y.min(),y.max())
dtopo.plot_dZ_colors(1.,axes=ax2)
fname = os.path.split(dtopo_fname)[-1] + '.png'
plt.savefig(fname)
print "Created ",fname

## Guardar alturas máximas y tiempos de arribo *on the run*

In [ ]:
from clawpack.geoclaw import fgmax_tools
fg = fgmax_tools.FGmaxGrid()
fg.point_style = 2       # will specify a 2d grid of points
fg.x1 = -120.
fg.x2 = -60.
fg.y1 = -60.
fg.y2 = 0.
fg.dx = 0.2 
fg.tstart_max =  10.     # when to start monitoring max values
fg.tend_max = 1.e10       # when to stop monitoring max values
fg.dt_check = 60.         # target time (sec) increment between updating 
                           # max values
fg.min_level_check = 2    # which levels to monitor max on
fg.arrival_tol = 1.e-2    # tolerance for flagging arrival

fg.input_file_name = 'fgmax_grid.txt'
fg.write_input_data()

## Configurar la simulación

Para correr la simulación necesitamos obligatoriamente tres archivos en la carpeta de la simulación (en este caso, de este cuaderno de *Jupyter*). Estos son: un buen *Makefile*, un *setrun.py* y opcionalmente un *setplot.py*. Normalmente, a menos que sepas muy bien lo que haces, NO edites el *Makefile* pero no te olvides de colocarlo en el mismo directorio que este cuaderno. El *setrun.py* por otro lado es el que definirá todos los parámetros de la simulación, y creará nuevos archivos que permitirán vincular el programa ejecutable con los datos de la simulación (topografía, falla y otros parámetros del setrun.py). El archivo ejecutable se crea y ejecuta usando el *Makefile*, como veremos en la siguiente sección.

El archivo setrun.py es muy largo y evidencia la complejidad que tiene este tipo de simulación. La mayor parte de los parámetros aparecen explicados [aquí](http://www.clawpack.org/setrun.html). Las que nos interesa editar a nosotros son las siguientes


* Extensión del dominio computacional, y resolución del nivel más grueso.

        # ---------------
        # Spatial domain:
        # ---------------

        ...

* Instantes de la simulación donde guardar resultados (cuidado, los archivos pueden volverse muy grandes con facilidad).

        # -------------
        # Output times:
        #--------------

        ...

* Configuración del paso de tiempo de la simulación. Normalmente sólo interesa modificar los parámetros que dicen "CFL", en caso que el cálculo "explote".

        # --------------
        # Time stepping:
        # --------------

        ...


* Condiciones de borde. Sólo existen bordes cerrado y abierto(extrap) ya programadas. Bordes más complejos deben ser programados en fortran por el usuario.

        # --------------------
        # Boundary conditions:
        # --------------------

        ...


* Parámetros del **refinamiento adaptativo ** y posición de las **boyas virtuales**.
        # ---------------
        # AMR parameters:
        # ---------------   

        ...
    
* y además, la que es exclusiva de GeoClaw, y donde se registr la topografía, falla y otros parámetros geofísicos como la fricción con el fondo, el sistema de coordenadas, entre otros.

        #-------------------
        def setgeo(rundata):
        #-------------------    
        ...

### Sobre el WorkFlow y algunos comandos mágicos de IPython

Recomiendo leer una vez con cuidado el setrun.py para entenderlo, pero luego usar el atajo de búsqueda CTRL+F/CMD+F para encontrar la sección  (como las que acabo de escribir, por ejemplo) o parámetro en particular que se desea modificar. 

Además, para crear el archivo setrun.py estoy ocupando la función mágica

    %%writefile setrun.py
    
que permite escribir el contenido de la celda en el archivo setrun.py. Ejecuta la celda a continuación para leer la ayuda disponible para esta función.

In [ ]:
%%writefile?

También puedes ver otras funciones mágicas usando

    %lsmagic

In [ ]:
%lsmagic

In [ ]:
%%writefile setrun.py
# %load setrun.py
"""
Module to set up run time parameters for Clawpack.

The values set in the function setrun are then written out to data files
that will be read in by the Fortran code.

"""

import os
import numpy as np

try:
    CLAW = os.environ['CLAW']
except:
    raise Exception("*** Must first set CLAW enviornment variable")

# Scratch directory for storing topo and dtopo files:
scratch_dir = os.path.join(CLAW, 'geoclaw', 'scratch')


#------------------------------
def setrun(claw_pkg='geoclaw'):
#------------------------------

    """
    Define the parameters used for running Clawpack.

    INPUT:
        claw_pkg expected to be "geoclaw" for this setrun.

    OUTPUT:
        rundata - object of class ClawRunData

    """

    from clawpack.clawutil import data

    assert claw_pkg.lower() == 'geoclaw',  "Expected claw_pkg = 'geoclaw'"

    num_dim = 2
    rundata = data.ClawRunData(claw_pkg, num_dim)


    #------------------------------------------------------------------
    # Problem-specific parameters to be written to setprob.data:
    #------------------------------------------------------------------
    
    #probdata = rundata.new_UserData(name='probdata',fname='setprob.data')


    #------------------------------------------------------------------
    # GeoClaw specific parameters:
    #------------------------------------------------------------------
    rundata = setgeo(rundata)

    #------------------------------------------------------------------
    # Standard Clawpack parameters to be written to claw.data:
    #   (or to amr2ez.data for AMR)
    #------------------------------------------------------------------
    clawdata = rundata.clawdata  # initialized when rundata instantiated


    # Set single grid parameters first.
    # See below for AMR parameters.


    # ---------------
    # Spatial domain:
    # ---------------

    # Number of space dimensions:
    clawdata.num_dim = num_dim

    # Lower and upper edge of computational domain:
    clawdata.lower[0] = -120.0      # west longitude
    clawdata.upper[0] = -60.0       # east longitude

    clawdata.lower[1] = -60.0       # south latitude
    clawdata.upper[1] = 0.0         # north latitude



    # Number of grid cells: Coarsest grid
    clawdata.num_cells[0] = 30
    clawdata.num_cells[1] = 30

    # ---------------
    # Size of system:
    # ---------------

    # Number of equations in the system:
    clawdata.num_eqn = 3

    # Number of auxiliary variables in the aux array (initialized in setaux)
    clawdata.num_aux = 4

    # Index of aux array corresponding to capacity function, if there is one:
    clawdata.capa_index = 2

    
    
    # -------------
    # Initial time:
    # -------------

    clawdata.t0 = 0.0


    # Restart from checkpoint file of a previous run?
    # Note: If restarting, you must also change the Makefile to set:
    #    RESTART = True
    # If restarting, t0 above should be from original run, and the
    # restart_file 'fort.chkNNNNN' specified below should be in 
    # the OUTDIR indicated in Makefile.

    clawdata.restart = False               # True to restart from prior results
    clawdata.restart_file = 'fort.chk00036'  # File to use for restart data

    # -------------
    # Output times:
    #--------------

    # Specify at what times the results should be written to fort.q files.
    # Note that the time integration stops after the final output time.
    # The solution at initial time t0 is always written in addition.

    clawdata.output_style = 1

    if clawdata.output_style==1:
        # Output nout frames at equally spaced times up to tfinal:
        clawdata.num_output_times = 5
        clawdata.tfinal = 10*3600.0
        clawdata.output_t0 = False  # output at initial (or restart) time?

    elif clawdata.output_style == 2:
        # Specify a list of output times.
        clawdata.output_times = [30., 60., 300., 600.]

    elif clawdata.output_style == 3:
        # Output every iout timesteps with a total of ntot time steps:
        clawdata.output_step_interval = 1
        clawdata.total_steps = 3
        clawdata.output_t0 = True
        

    clawdata.output_format = 'ascii'      # 'ascii' or 'netcdf' 

    clawdata.output_q_components = 'all'   # need all
    clawdata.output_aux_components = 'none'  # eta=h+B is in q
    clawdata.output_aux_onlyonce = False    # output aux arrays each frame



    # ---------------------------------------------------
    # Verbosity of messages to screen during integration:
    # ---------------------------------------------------

    # The current t, dt, and cfl will be printed every time step
    # at AMR levels <= verbosity.  Set verbosity = 0 for no printing.
    #   (E.g. verbosity == 2 means print only on levels 1 and 2.)
    clawdata.verbosity = 1



    # --------------
    # Time stepping:
    # --------------

    # if dt_variable==1: variable time steps used based on cfl_desired,
    # if dt_variable==0: fixed time steps dt = dt_initial will always be used.
    clawdata.dt_variable = True

    # Initial time step for variable dt.
    # If dt_variable==0 then dt=dt_initial for all steps:
    clawdata.dt_initial = 0.2

    # Max time step to be allowed if variable dt used:
    clawdata.dt_max = 1e+99

    # Desired Courant number if variable dt used, and max to allow without
    # retaking step with a smaller dt:
    clawdata.cfl_desired = 0.75
    clawdata.cfl_max = 1.0

    # Maximum number of time steps to allow between output times:
    clawdata.steps_max = 5000




    # ------------------
    # Method to be used:
    # ------------------

    # Order of accuracy:  1 => Godunov,  2 => Lax-Wendroff plus limiters
    clawdata.order = 2
    
    # Use dimensional splitting? (not yet available for AMR)
    clawdata.dimensional_split = 'unsplit'
    
    # For unsplit method, transverse_waves can be 
    #  0 or 'none'      ==> donor cell (only normal solver used)
    #  1 or 'increment' ==> corner transport of waves
    #  2 or 'all'       ==> corner transport of 2nd order corrections too
    clawdata.transverse_waves = 2

    # Number of waves in the Riemann solution:
    clawdata.num_waves = 3
    
    # List of limiters to use for each wave family:  
    # Required:  len(limiter) == num_waves
    # Some options:
    #   0 or 'none'     ==> no limiter (Lax-Wendroff)
    #   1 or 'minmod'   ==> minmod
    #   2 or 'superbee' ==> superbee
    #   3 or 'mc'       ==> MC limiter
    #   4 or 'vanleer'  ==> van Leer
    clawdata.limiter = ['mc', 'mc', 'mc']

    clawdata.use_fwaves = True    # True ==> use f-wave version of algorithms
    
    # Source terms splitting:
    #   src_split == 0 or 'none'    ==> no source term (src routine never called)
    #   src_split == 1 or 'godunov' ==> Godunov (1st order) splitting used, 
    #   src_split == 2 or 'strang'  ==> Strang (2nd order) splitting used,  not recommended.
    clawdata.source_split = 'godunov'


    # --------------------
    # Boundary conditions:
    # --------------------

    # Number of ghost cells (usually 2)
    clawdata.num_ghost = 2

    # Choice of BCs at xlower and xupper:
    #   0 => user specified (must modify bcN.f to use this option)
    #   1 => extrapolation (non-reflecting outflow)
    #   2 => periodic (must specify this at both boundaries)
    #   3 => solid wall for systems where q(2) is normal velocity

    clawdata.bc_lower[0] = 'extrap'
    clawdata.bc_upper[0] = 'extrap'

    clawdata.bc_lower[1] = 'extrap'
    clawdata.bc_upper[1] = 'extrap'



    # --------------
    # Checkpointing:
    # --------------

    # Specify when checkpoint files should be created that can be
    # used to restart a computation.

    clawdata.checkpt_style = 0

    if clawdata.checkpt_style == 0:
        # Do not checkpoint at all
        pass

    elif clawdata.checkpt_style == 1:
        # Checkpoint only at tfinal.
        pass

    elif clawdata.checkpt_style == 2:
        # Specify a list of checkpoint times.  
        clawdata.checkpt_times = [0.1,0.15]

    elif clawdata.checkpt_style == 3:
        # Checkpoint every checkpt_interval timesteps (on Level 1)
        # and at the final time.
        clawdata.checkpt_interval = 5


    # ---------------
    # AMR parameters:
    # ---------------
    amrdata = rundata.amrdata

    # max number of refinement levels:
    amrdata.amr_levels_max = 2

    # List of refinement ratios at each level (length at least mxnest-1)
    amrdata.refinement_ratios_x = [4,3]
    amrdata.refinement_ratios_y = [4,3]
    amrdata.refinement_ratios_t = [4,3]


    # Specify type of each aux variable in amrdata.auxtype.
    # This must be a list of length maux, each element of which is one of:
    #   'center',  'capacity', 'xleft', or 'yleft'  (see documentation).

    amrdata.aux_type = ['center','capacity','yleft','center']


    # Flag using refinement routine flag2refine rather than richardson error
    amrdata.flag_richardson = False    # use Richardson?
    amrdata.flag2refine = True

    # steps to take on each level L between regriddings of level L+1:
    amrdata.regrid_interval = 3

    # width of buffer zone around flagged points:
    # (typically the same as regrid_interval so waves don't escape):
    amrdata.regrid_buffer_width  = 2

    # clustering alg. cutoff for (# flagged pts) / (total # of cells refined)
    # (closer to 1.0 => more small grids may be needed to cover flagged cells)
    amrdata.clustering_cutoff = 0.700000

    # print info about each regridding up to this level:
    amrdata.verbosity_regrid = 0  

    #  ----- For developers ----- 
    # Toggle debugging print statements:
    amrdata.dprint = False      # print domain flags
    amrdata.eprint = False      # print err est flags
    amrdata.edebug = False      # even more err est flags
    amrdata.gprint = False      # grid bisection/clustering
    amrdata.nprint = False      # proper nesting output
    amrdata.pprint = False      # proj. of tagged points
    amrdata.rprint = False      # print regridding summary
    amrdata.sprint = False      # space/memory output
    amrdata.tprint = True       # time step reporting each level
    amrdata.uprint = False      # update/upbnd reporting
    
    # More AMR parameters can be set -- see the defaults in pyclaw/data.py

    # ---------------
    # Regions:
    # ---------------
    rundata.regiondata.regions = []
    # to specify regions of refinement append lines of the form
    #  [minlevel,maxlevel,t1,t2,x1,x2,y1,y2]
    #rundata.regiondata.regions.append([1, 2, 0., 1e9, -360,360,-90,90])
    rundata.regiondata.regions.append([3, 3, 0., 600., -76,-72,-38,-33])

    # ---------------
    # Gauges:
    # ---------------
    rundata.gaugedata.gauges = []
    # for gauges append lines of the form  [gaugeno, x, y, t1, t2]
    rundata.gaugedata.gauges.append([32412, -86.392, -17.975, 0., 1.e10])
    

    return rundata
    # end of function setrun
    # ----------------------


#-------------------
def setgeo(rundata):
#-------------------
    """
    Set GeoClaw specific runtime parameters.
    For documentation see ....
    """

    try:
        geo_data = rundata.geo_data
    except:
        print "*** Error, this rundata has no geo_data attribute"
        raise AttributeError("Missing geo_data attribute")
       
    # == Physics ==
    geo_data.gravity = 9.81
    geo_data.coordinate_system = 2
    geo_data.earth_radius = 6367.5e3

    # == Forcing Options
    geo_data.coriolis_forcing = False

    # == Algorithm and Initial Conditions ==
    geo_data.sea_level = 0.0
    geo_data.dry_tolerance = 1.e-3
    geo_data.friction_forcing = True
    geo_data.manning_coefficient =.025
    geo_data.friction_depth = 1e6

    # Refinement settings
    refinement_data = rundata.refinement_data
    refinement_data.variable_dt_refinement_ratios = True
    refinement_data.wave_tolerance = 1.e-2
    refinement_data.deep_depth = 1e2
    refinement_data.max_level_deep = 3

    # == settopo.data values ==
    topo_data = rundata.topo_data
    # for topography, append lines of the form
    #    [topotype, minlevel, maxlevel, t1, t2, fname]
    topo_path = os.path.join(scratch_dir, 'etopo10min120W60W60S0S.asc')
    topo_data.topofiles.append([2, 1, 3, 0., 1.e10, topo_path])

    # == setdtopo.data values ==
    dtopo_data = rundata.dtopo_data
    # for moving topography, append lines of the form :   (<= 1 allowed for now!)
    #   [topotype, minlevel,maxlevel,fname]
    dtopo_path = os.path.join(scratch_dir, 'dtopo_usgs100227.tt3')
    dtopo_data.dtopofiles.append([3,3,3,dtopo_path])
    dtopo_data.dt_max_dtopo = 0.2


    # == setqinit.data values ==
    rundata.qinit_data.qinit_type = 0
    rundata.qinit_data.qinitfiles = []
    # for qinit perturbations, append lines of the form: (<= 1 allowed for now!)
    #   [minlev, maxlev, fname]

    # == setfixedgrids.data values ==
    fixed_grids = rundata.fixed_grid_data
    # for fixed grids append lines of the form
    # [t1,t2,noutput,x1,x2,y1,y2,xpoints,ypoints,\
    #  ioutarrivaltimes,ioutsurfacemax]

    # == fgmax.data values ==
    fgmax_files = rundata.fgmax_data.fgmax_files
    # for fixed grids append to this list names of any fgmax input files
    fgmax_files.append('fgmax_grid.txt')
    rundata.fgmax_data.num_fgmax_val = 1  # Save depth only



    return rundata
    # end of function setgeo
    # ----------------------



if __name__ == '__main__':
    # Set up run-time parameters and write all data files.
    import sys
    rundata = setrun(*sys.argv[1:])
    rundata.write()



## Correr la simulación

En el archivo *Makefile* que está en este directorio se encuentran definidos comandos que permiten compilar y ejecutar el programa con los datos de la simulación. Normalmente se ejecutaría desde un terminal o [línea de comandos](https://en.wikipedia.org/wiki/Command-line_interface). En el cuaderno de *Jupyter* basta anteponer el signo "!" en una celda de código.

Por ejemplo, ejecutando la línea

    !make help

lista los comandos que hay definidos en el *Makefile* actual.

In [ ]:
!make help

Para compilar y ejecutar el programa usamos

    make .output
    
Verás que en la celda se imprimirá información sobre el progreso de la simulación.

In [ ]:
!make clean
!make .output

Utilizando dos niveles de refinamiento (busca con ctrĺ+f: *amrdata.amr_levels_max = 2*), en mi computador tomó 55.155s.

## Procesando los resultados de la simulación

### Mapa de propagación: alturas máximas y tiempos de arribo

In [ ]:
from clawpack.geoclaw import fgmax_tools, geoplot

Read fgmax setup file and results.

In [ ]:
fg = fgmax_tools.FGmaxGrid()
fg.read_input_data('fgmax_grid.txt')
fg.read_output()

In [ ]:
clines_zeta = [0.01] + list(np.linspace(0.05,0.3,6)) + [0.5,1.0,10.0]
colors = geoplot.discrete_cmap_1(clines_zeta)


plt.figure(figsize=(12,6))
plt.clf()
zeta = np.where(fg.B>0, fg.h, fg.h+fg.B)   # surface elevation in ocean
plt.contourf(fg.X,fg.Y,zeta,clines_zeta,colors=colors)
plt.colorbar()
plt.contour(fg.X,fg.Y,fg.B,[0.],colors='k')  # coastline

# plot arrival time contours and label:
arrival_t = fg.arrival_time/3600.  # arrival time in hours
clines_t = np.linspace(0,8,17)  # hours
clines_t_label = clines_t[::2]  # which ones to label 
clines_t_colors = ([.5,.5,.5],)
con_t = plt.contour(fg.X,fg.Y,arrival_t, clines_t,colors=clines_t_colors) 
plt.clabel(con_t, clines_t_label)

# fix axes:
plt.ticklabel_format(format='plain',useOffset=False)
plt.xticks(rotation=20)
plt.gca().set_aspect(1./np.cos(fg.Y.mean()*np.pi/180.))
plt.title("Maximum amplitude [m] / arrival times [h]")

### Creando una animación de la propagación

In [ ]:
%%writefile setplot.py
# %load setplot.py

""" 
Set up the plot figures, axes, and items to be done for each frame.

This module is imported by the plotting routines and then the
function setplot is called to set the plot parameters.
    
""" 

import numpy as np
import matplotlib.pyplot as plt

from clawpack.geoclaw import topotools

try:
    TG32412 = np.loadtxt('32412_notide.txt')
except:
    print "*** Could not load DART data file"

#--------------------------
def setplot(plotdata):
#--------------------------
    
    """ 
    Specify what is to be plotted at each frame.
    Input:  plotdata, an instance of pyclaw.plotters.data.ClawPlotData.
    Output: a modified version of plotdata.
    
    """ 


    from clawpack.visclaw import colormaps, geoplot
    from numpy import linspace

    plotdata.clearfigures()  # clear any old figures,axes,items data


    # To plot gauge locations on pcolor or contour plot, use this as
    # an afteraxis function:

    def addgauges(current_data):
        from clawpack.visclaw import gaugetools
        gaugetools.plot_gauge_locations(current_data.plotdata, \
             gaugenos='all', format_string='ko', add_labels=True)
    

    #-----------------------------------------
    # Figure for surface
    #-----------------------------------------
    plotfigure = plotdata.new_plotfigure(name='Surface', figno=0)

    # Set up for axes in this figure:
    plotaxes = plotfigure.new_plotaxes('pcolor')
    plotaxes.title = 'Surface'
    plotaxes.scaled = True

    def fixup(current_data):
        import pylab
        addgauges(current_data)
        t = current_data.t
        t = t / 3600.  # hours
        pylab.title('Surface at %4.2f hours' % t, fontsize=20)
        pylab.xticks(fontsize=15)
        pylab.yticks(fontsize=15)
    plotaxes.afteraxes = fixup

    # Water
    plotitem = plotaxes.new_plotitem(plot_type='2d_pcolor')
    #plotitem.plot_var = geoplot.surface
    plotitem.plot_var = geoplot.surface_or_depth
    plotitem.pcolor_cmap = geoplot.tsunami_colormap
    plotitem.pcolor_cmin = -0.2
    plotitem.pcolor_cmax = 0.2
    plotitem.add_colorbar = True
    plotitem.amr_celledges_show = [0,0,0]
    plotitem.patchedges_show = 1

    # Land
    plotitem = plotaxes.new_plotitem(plot_type='2d_pcolor')
    plotitem.plot_var = geoplot.land
    plotitem.pcolor_cmap = geoplot.land_colors
    plotitem.pcolor_cmin = 0.0
    plotitem.pcolor_cmax = 100.0
    plotitem.add_colorbar = False
    plotitem.amr_celledges_show = [1,0,0]
    plotitem.patchedges_show = 1
    plotaxes.xlimits = [-120,-60]
    plotaxes.ylimits = [-60,0]

    # add contour lines of bathy if desired:
    plotitem = plotaxes.new_plotitem(plot_type='2d_contour')
    plotitem.show = False
    plotitem.plot_var = geoplot.topo
    plotitem.contour_levels = linspace(-3000,-3000,1)
    plotitem.amr_contour_colors = ['y']  # color on each level
    plotitem.kwargs = {'linestyles':'solid','linewidths':2}
    plotitem.amr_contour_show = [1,0,0]  
    plotitem.celledges_show = 0
    plotitem.patchedges_show = 0


    #-----------------------------------------
    # Figures for gauges
    #-----------------------------------------
    plotfigure = plotdata.new_plotfigure(name='Surface at gauges', figno=300, \
                    type='each_gauge')
    plotfigure.clf_each_gauge = True

    # Set up for axes in this figure:
    plotaxes = plotfigure.new_plotaxes()
    plotaxes.xlimits = 'auto'
    plotaxes.ylimits = 'auto'
    plotaxes.title = 'Surface'

    # Plot surface as blue curve:
    plotitem = plotaxes.new_plotitem(plot_type='1d_plot')
    plotitem.plot_var = 3
    plotitem.plotstyle = 'b-'

    # Plot topo as green curve:
    plotitem = plotaxes.new_plotitem(plot_type='1d_plot')
    plotitem.show = False

    def gaugetopo(current_data):
        q = current_data.q
        h = q[0,:]
        eta = q[3,:]
        topo = eta - h
        return topo
        
    plotitem.plot_var = gaugetopo
    plotitem.plotstyle = 'g-'

    def add_zeroline(current_data):
        from pylab import plot, legend, xticks, floor, axis, xlabel
        t = current_data.t 
        gaugeno = current_data.gaugeno

        if gaugeno == 32412:
            try:
                plot(TG32412[:,0], TG32412[:,1], 'r')
                legend(['GeoClaw','Obs'],'lower right')
            except: pass
            axis((0,t.max(),-0.3,0.3))

        plot(t, 0*t, 'k')
        n = int(floor(t.max()/3600.) + 2)
        xticks([3600*i for i in range(n)], ['%i' % i for i in range(n)])
        xlabel('time (hours)')

    plotaxes.afteraxes = add_zeroline


    #-----------------------------------------
    # Figures for fgmax - max values on fixed grids
    #-----------------------------------------
    otherfigure = plotdata.new_otherfigure(name='max amplitude and arrival times', 
                    fname='amplitude_times.png')



    #-----------------------------------------
    
    # Parameters used only when creating html and/or latex hardcopy
    # e.g., via pyclaw.plotters.frametools.printframes:

    plotdata.printfigs = True                # print figures
    plotdata.print_format = 'png'            # file format
    plotdata.print_framenos = 'all'          # list of frames to print
    plotdata.print_gaugenos = 'all'          # list of gauges to print
    plotdata.print_fignos = 'all'            # list of figures to print
    plotdata.html = True                     # create html files of plots?
    plotdata.html_homelink = '../README.html'   # pointer for top of index
    plotdata.latex = True                    # create latex file of plots?
    plotdata.latex_figsperline = 2           # layout of plots
    plotdata.latex_framesperline = 1         # layout of plots
    plotdata.latex_makepdf = False           # also run pdflatex?

    return plotdata



In [ ]:
!make plots

# The animation

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame('_plots/movieframe_allframesfig0.html',width=600, height=600)

## The gauge

In [ ]:
from IPython.display import Image
Image('_plots/gauge32412fig300.png', width=600)

In [7]:
from IPython.core.display import HTML
css_file = 'style/style.css'
HTML(open(css_file, "r").read())